<a href="https://colab.research.google.com/github/Shyam-duba/NewsArticlesClassification/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00


In [5]:
from transformers import DistilBertTokenizerFast,TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np


In [11]:
path = "/all-data.csv"

# New Section

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv(path,encoding='latin-1')
df.head()

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [118]:
df['sentiment'] = df['neutral']
df['text'] = df['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .']
df

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",sentiment,text
0,neutral,Technopolis plans to develop in stages an area...,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,negative,LONDON MarketWatch -- Share prices ended lower...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4843,negative,Net sales of the Paper segment decreased to EU...,negative,Net sales of the Paper segment decreased to EU...


In [119]:
df.drop(['neutral','According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .'],axis=1)
df['sentiment'] = df['sentiment'].map({'positive':0,'neutral':1,'negative':2})
df

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",sentiment,text
0,neutral,Technopolis plans to develop in stages an area...,1,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...,2,The international electronic industry company ...
2,positive,With the new production plant the company woul...,0,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...,0,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,2,LONDON MarketWatch -- Share prices ended lower...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,2,Operating profit fell to EUR 35.4 mn from EUR ...
4843,negative,Net sales of the Paper segment decreased to EU...,2,Net sales of the Paper segment decreased to EU...


In [120]:
text = df['text'].values.tolist()
semt = df['sentiment'].tolist()

In [121]:
from sklearn.model_selection import train_test_split
train_text, test_text, train_semt,test_semt = train_test_split(text,semt,test_size=0.2)


In [122]:
tokenzier = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [123]:
tokenzier([train_text[0]],truncation=True,padding=True,max_length=128)

{'input_ids': [[101, 1037, 27902, 2449, 2038, 2623, 2009, 2003, 29396, 1037, 2658, 17493, 2447, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [124]:
train_encoding = tokenzier(train_text,truncation=True,padding=True,max_length=128)
test_encoding = tokenzier(test_text,truncation=True,padding=True,max_length=128)

In [125]:
train_data = tf.data.Dataset.from_tensor_slices((dict(train_encoding),train_semt))

In [126]:
test_data = tf.data.Dataset.from_tensor_slices((dict(test_encoding),test_semt))

In [127]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [128]:
optmiz = tf.keras.optimizers.Adam(learning_rate=5e-5,epsilon=1e-8)
model.compile(optimizer=optmiz,loss=model.compute_loss,metrics=['accuracy'])

In [138]:
sample = 'shyam loves Robin'

In [139]:
predict_input = tokenzier.encode(sample,truncation=True,padding=True)

In [140]:
tf_out = model.predict(predict_input)[0]
tf_predict = tf.nn.softmax(tf_out,axis=1)
labels = ['positive','neutral','negative']
label = tf.argmax(tf_predict,axis=1)
label = label.numpy()
print(labels[label[0]])

1/1 [==============================] - 0s 76ms/step
positive
